# Analysis of intraday storage

Using the base case hydrology and P&E modelling, determine the maximum volume of water that can be held back during two 10 hour storage periods and then released during two 2 hour peaking periods. 

Calculate the energy transferred from the storage (off-peak) to peak periods.

Calculate the rise in the headpond and tailrace during each storage period (and therefore fall during each peak period). 

### Setup

In [726]:
import numpy as np
import pandas as pd
import datetime
import math

In [727]:
def tailwaterLevel(flow):
    if fixed_tailwater_level>0:
        return fixed_tailwater_level
    else:
        log_flow=math.log10(flow)
        if log_flow>=3.8:
            return -21.429*log_flow**2+182.03*log_flow+597.15   
        elif log_flow>=2.3:
            return 6.5299*log_flow**2-30.155*log_flow+999.52   
        else:
            return -2.6828*log_flow**2+11.265*log_flow+952.68

In [728]:
flow_data='./output_data/ngonye_daily.csv'
energy_data='./output_data/base_pe_daily.csv'

canal_capacity=1100
rated_flow_unit=250

#offpeak_min_flow=rated_flow_unit * 0.5
offpeak_min_flow=50
#offpeak_min_flow=rated_flow_unit

storage_curve=10 # 10 Mm3/m headpond rise - From Feasibility Study Rev D Appendix H - Mott MacDonald
#max_power=185 # Max output for whole plant
max_power=205
fixed_tailwater_level = 0

night_hours = 10 # Night off-peak
morning_hours = 3 # Length of morning peak
standard_hours = 8 # Time between morning peak and evening peak when storing can occur
evening_hours = 3 # Length of evening peak

### Load data

In [729]:
flow = pd.read_csv(flow_data).set_index('Date')
flow.columns

Index(['LaggedDate', 'VicFalls', 'Conversion', 'Flow', 'Exceedance', 'Year',
       'Month', 'Day', 'MonthId', 'WaterYear', 'WaterMonth', 'WaterDay',
       'WaterWeek', 'Volume', 'Flow_difference', 'Flow_difference_pct',
       'EWRRefExceedance'],
      dtype='object')

In [730]:
pe = pd.read_csv(energy_data).set_index('Date')
pe.columns

Index(['LaggedDate', 'VicFalls', 'Conversion', 'Flow', 'Exceedance', 'Year',
       'Month', 'Day', 'MonthId', 'WaterYear', 'WaterMonth', 'WaterDay',
       'WaterWeek', 'Volume', 'Flow_difference', 'Flow_difference_pct',
       'EWRRefExceedance', 'EWRBandNo', 'EWRBandLabel', 'EWRChannelA',
       'EWRChannelC', 'EWRChannelD', 'EWRChannelE', 'EWRChannelFG', 'EWRTotal',
       'EWRProportion', 'FlowAvailableForGeneration', 'FlowCanal',
       'LowFlowShutoff', 'Turbines', 'FlowTurbine1', 'FlowTurbine2',
       'FlowTurbine3', 'FlowTurbine4', 'FlowSpill', 'SpillChannelA',
       'SpillChannelC', 'SpillChannelD', 'SpillChannelE', 'SpillChannelFG',
       'FlowChannelA', 'FlowChannelC', 'FlowChannelD', 'FlowChannelE',
       'FlowChannelFG', 'FlowLeftChannel', 'LevelTailwater', 'LevelHeadpond',
       'HeadlossLeftChannel', 'HeadlossCanal', 'LevelForebay',
       'HeadlossTurbine1', 'HeadlossTurbine2', 'HeadlossTurbine3',
       'HeadlossTurbine4', 'HeadTurbine1', 'HeadTurbine2', 'HeadTur

In [731]:
effective_efficiency = pe['EffTurbine1'].quantile(0.8)*pe['EffGen1'].quantile(0.8)

In [732]:
tmp=pe[['LowHeadShutoff','EWRTotal','FlowSpill','FlowAvailableForGeneration','FlowCanal','Power','Energy','LevelHeadpond','LevelTailwater', 'HeadTurbine1']].copy()
tmp['Flow']=flow['Flow']
tmp['Exceedance']=flow['Exceedance']

tmp['LowHeadShutoff'] = tmp['LowHeadShutoff'].fillna(0)
tmp.columns

Index(['LowHeadShutoff', 'EWRTotal', 'FlowSpill', 'FlowAvailableForGeneration',
       'FlowCanal', 'Power', 'Energy', 'LevelHeadpond', 'LevelTailwater',
       'HeadTurbine1', 'Flow', 'Exceedance'],
      dtype='object')

In [733]:
tmp['ExceedanceRnd']=np.round(tmp['Exceedance']*2,2)/2
work=tmp.reset_index().groupby('ExceedanceRnd').mean().drop(['Exceedance'],axis=1)
work

,LowHeadShutoff,EWRTotal,FlowSpill,FlowAvailableForGeneration,FlowCanal,Power,Energy,LevelHeadpond,LevelTailwater,HeadTurbine1,Flow
ExceedanceRnd,,,,,,,,,,,
0.000,1.00000,276.632353,8588.543600,8588.543600,0.000000,0.000000,0.000000,990.527642,981.764386,6.826354,8865.175953
0.005,1.00000,324.517241,6154.272848,6154.272848,0.000000,0.000000,0.000000,990.128209,979.484350,8.706957,6478.790089
0.010,0.56213,321.165680,4741.890222,5223.547027,481.656805,38.471131,923.307133,990.050824,978.148915,9.965006,5544.712707
0.015,0.00000,305.947674,3785.016338,4885.016338,1100.000000,91.548338,2197.160113,990.022062,977.618191,10.466969,5190.964012
0.020,0.00000,302.523256,3375.069915,4475.069915,1100.000000,98.867101,2372.810412,990.000000,976.964195,11.098903,4777.593171
...,...,...,...,...,...,...,...,...,...,...,...
0.980,0.00000,64.833333,0.000000,120.810563,120.810563,26.263488,630.323719,990.000000,964.428484,25.372253,185.643897
0.985,0.00000,60.697115,0.000000,114.563550,114.563550,24.754930,594.118324,990.000000,964.449454,25.365246,175.260665
0.990,0.00000,56.600000,0.000000,108.425295,108.425295,23.264526,558.348614,990.000000,964.467851,25.359957,165.025295


### Peak Periods Requirements for maximum output

In [734]:
#Total generation volume Mm3
work['GenerationVol'] = work['FlowCanal'] * (24 * 60 * 60) / 1e+06

#Rough calculation of the maximum flow during peak time - flow that produced maximum output at current head and high efficiency
work['MaxGenerationFlow'] = max_power / work['HeadTurbine1'] / 9.81 / effective_efficiency * 1000
#Limit maximum flow to canal capacity
work['MaxGenerationFlow'] = np.minimum(work['MaxGenerationFlow'],canal_capacity)

#Maximum flow that could be used by the peaks from storage m3/s
work['MaxFlowFromStore'] = work['MaxGenerationFlow'] - work['FlowCanal']

#Max volume that could be used by the peaks Mm3
work['MorningMaxVolFromStore'] = work['MaxFlowFromStore'] * (morning_hours * 60 * 60) / 1e+06
work['EveningMaxVolFromStore'] = work['MaxFlowFromStore'] * (evening_hours * 60 * 60) / 1e+06


### Max volume which the off-peak (night) and standard (day) periods can store

In [735]:
#Max volume that could be taken from the night period for storage Mm3
work['NightMaxVolToStore'] = ((work['FlowAvailableForGeneration'] - offpeak_min_flow) * (night_hours * 60 * 60) / 1e+06).clip(lower=0)

#Max volume that could be taken from the standard period for storage Mm3
work['StandardMaxVolToStore'] = ((work['FlowAvailableForGeneration'] - offpeak_min_flow) * (standard_hours * 60 * 60) / 1e+06).clip(lower=0)


### Storage volumes

In [736]:
#Volume per night that will be stored (min of available or needed)
work['NightStoreVol'] = work[['MorningMaxVolFromStore','NightMaxVolToStore']].min(axis=1)

#Not enough head for generation so no point storing
work.loc[work['LowHeadShutoff'] == 1.0, 'NightStoreVol'] = 0

#Volume per standard period that will be stored (min of available or needed)
work['StandardStoreVol'] = work[['EveningMaxVolFromStore','StandardMaxVolToStore']].min(axis=1)

#Not enough head for generation so no point storing
work.loc[work['LowHeadShutoff'] == 1.0, 'NightStoreVol'] = 0


### Flows

In [737]:
#Night
work['NightExtraFlow']=(work['NightStoreVol'] / (60 * 60 * night_hours / 1e+06)) # Flow diverted to storage during night
work['NightFlowCanal']=work['FlowCanal'] - work['NightExtraFlow'] #Resulting canal flow during off-peak

#Morning
work['MorningExtraFlow']=(work['NightStoreVol'] / (60 * 60 * morning_hours / 1e+06)) # Extra flow from storage during morning peak 
work['MorningFlowCanal']=work['FlowCanal'] + work['MorningExtraFlow'] #Resulting canal flow during morning peak

#Standard
work['StandardExtraFlow']=(work['StandardStoreVol'] / (60 * 60 * standard_hours / 1e+06)) # Flow diverted to storage during standard period
work['StandardFlowCanal']=work['FlowCanal'] - work['StandardExtraFlow'] #Resulting canal flow during standard period

#Evening
work['EveningExtraFlow']=(work['StandardStoreVol'] / (60 * 60 * evening_hours / 1e+06)) # Extra flow from storage during evening peak 
work['EveningFlowCanal']=work['FlowCanal'] + work['EveningExtraFlow'] #Resulting canal flow during evening peak


### Total flow

In [738]:
work['NightTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['NightFlowCanal']
work['MorningTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['MorningFlowCanal']
work['StandardTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['StandardFlowCanal']
work['EveningTotalFlow']=work['FlowSpill'] + work['EWRTotal'] + work['EveningFlowCanal']


### Total volumes

In [739]:
work['NightVol']=work['NightFlowCanal'] * night_hours * 60 * 60 / 1e+06
work['MorningVol']=work['MorningFlowCanal'] * morning_hours * 60 * 60 / 1e+06
work['StandardVol']=work['StandardFlowCanal'] * standard_hours * 60 * 60 / 1e+06
work['EveningVol']=work['EveningFlowCanal'] * evening_hours * 60 * 60 / 1e+06

### Energy

In [740]:
work['NightEnergy'] = work['Energy'] / work['GenerationVol'] * work['NightVol']
work['MorningEnergy'] = work['Energy'] / work['GenerationVol'] * work['MorningVol']
work['StandardEnergy'] = work['Energy'] / work['GenerationVol'] * work['StandardVol']
work['EveningEnergy'] = work['Energy'] / work['GenerationVol'] * work['EveningVol']

work['MorningEnergyTransfer'] = work['Energy'] / work['GenerationVol'] * work['NightStoreVol']
work['EveningEnergyTransfer'] = work['Energy'] / work['GenerationVol'] * work['StandardStoreVol']


### Power

In [741]:
work['NightPower'] = work['NightEnergy'] / night_hours
work['MorningPower'] = work['MorningEnergy'] / morning_hours
work['StandardPower'] = work['StandardEnergy'] / standard_hours
work['EveningPower'] = work['EveningEnergy'] / evening_hours

### Headpond levels

In [742]:
work['NightHeadpondChange']=work['NightStoreVol'] / storage_curve
work['StandardHeadpondChange']=work['StandardStoreVol'] / storage_curve

### Tailwater levels

In [743]:
work['NightTailwaterLevel']=work['NightTotalFlow'].apply(tailwaterLevel) 
work['MorningTailwaterLevel']=work['MorningTotalFlow'].apply(tailwaterLevel) 
work['StandardTailwaterLevel']=work['StandardTotalFlow'].apply(tailwaterLevel) 
work['EveningTailwaterLevel']=work['EveningTotalFlow'].apply(tailwaterLevel) 

work['NightTailwaterChange']=work['MorningTailwaterLevel'] - work['NightTailwaterLevel']
work['StandardTailwaterChange']=work['EveningTailwaterLevel'] - work['StandardTailwaterLevel']



In [744]:

work.loc[work['MorningMaxVolFromStore'] < work['NightMaxVolToStore'],'NightSupplyLimit'] = 'False'
work.loc[work['MorningMaxVolFromStore'] > work['NightMaxVolToStore'],'NightSupplyLimit'] = 'True'
work.loc[work['EveningMaxVolFromStore'] < work['StandardMaxVolToStore'],'StandardSupplyLimit'] = 'False'
work.loc[work['EveningMaxVolFromStore'] > work['StandardMaxVolToStore'],'StandardSupplyLimit'] = 'True'

work

,LowHeadShutoff,EWRTotal,FlowSpill,FlowAvailableForGeneration,FlowCanal,Power,Energy,LevelHeadpond,LevelTailwater,HeadTurbine1,...,NightHeadpondChange,StandardHeadpondChange,NightTailwaterLevel,MorningTailwaterLevel,StandardTailwaterLevel,EveningTailwaterLevel,NightTailwaterChange,StandardTailwaterChange,NightSupplyLimit,StandardSupplyLimit
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.000,1.00000,276.632353,8588.543600,8588.543600,0.000000,0.000000,0.000000,990.527642,981.764386,6.826354,...,0.000000,1.188000,981.792948,981.792948,981.518072,982.389895,0.000000,0.871823,False,False
0.005,1.00000,324.517241,6154.272848,6154.272848,0.000000,0.000000,0.000000,990.128209,979.484350,8.706957,...,0.000000,1.188000,979.646743,979.646743,978.892136,980.819361,0.000000,1.927224,False,False
0.010,0.56213,321.165680,4741.890222,5223.547027,481.656805,38.471131,923.307133,990.050824,978.148915,9.965006,...,0.667811,0.667811,977.875013,979.024743,977.805103,979.024743,1.149730,1.219641,False,False
0.015,0.00000,305.947674,3785.016338,4885.016338,1100.000000,91.548338,2197.160113,990.022062,977.618191,10.466969,...,0.000000,0.000000,977.619349,977.619349,977.619349,977.619349,0.000000,0.000000,False,False
0.020,0.00000,302.523256,3375.069915,4475.069915,1100.000000,98.867101,2372.810412,990.000000,976.964195,11.098903,...,0.000000,0.000000,976.965962,976.965962,976.965962,976.965962,0.000000,0.000000,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.980,0.00000,64.833333,0.000000,120.810563,120.810563,26.263488,630.323719,990.000000,964.428484,25.372253,...,0.254918,0.203934,964.501184,965.358143,964.501184,965.162719,0.856959,0.661534,True,True
0.985,0.00000,60.697115,0.000000,114.563550,114.563550,24.754930,594.118324,990.000000,964.449454,25.365246,...,0.232429,0.185943,964.497134,965.227625,964.497134,965.057225,0.730492,0.560091,True,True
0.990,0.00000,56.600000,0.000000,108.425295,108.425295,23.264526,558.348614,990.000000,964.467851,25.359957,...,0.210331,0.168265,964.491550,965.104650,964.491550,964.960271,0.613100,0.468722,True,True


In [745]:
work.loc[0.640]

LowHeadShutoff                       0
EWRTotal                       129.814
FlowSpill                            0
FlowAvailableForGeneration     283.053
FlowCanal                      283.053
Power                          60.6687
Energy                         1456.05
LevelHeadpond                      990
LevelTailwater                 965.321
HeadTurbine1                    24.247
Flow                           412.867
GenerationVol                  24.4558
MaxGenerationFlow              935.563
MaxFlowFromStore                652.51
MorningMaxVolFromStore         7.04711
EveningMaxVolFromStore         7.04711
NightMaxVolToStore             8.38991
StandardMaxVolToStore          6.71193
NightStoreVol                  7.04711
StandardStoreVol               6.71193
NightExtraFlow                 195.753
NightFlowCanal                    87.3
MorningExtraFlow                652.51
MorningFlowCanal               935.563
StandardExtraFlow              233.053
StandardFlowCanal        

In [746]:
work.to_csv('./intraday/intraday_50cumec_220mw.csv')